In [1]:
from flask import Flask, jsonify,request
import pickle
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import base64
from io import BytesIO
from rembg import remove

In [2]:
MODEL_PATH="C:/Users/USUARIO/Desktop/Tesis/gitrepo/cocoa_dl_android/FlaskWebService2/src/"

In [3]:
# Load Model
#==============================================================================

Model_json = MODEL_PATH+"model.json"
#Model_weights = MODEL_PATH+"model.h5"

model_json = open(Model_json, 'r')
loaded_model_json = model_json.read()
model_json.close()
# model = tf.keras.models.model_from_json(loaded_model_json)
# model.load_weights(Model_weights)

models_path=["model_fito.h5","model_mazorca_negra.h5","model_monoliosis_ef.h5","model_monoliosis_intermedia_sf.h5"]
models=[]

for path in models_path:
    Model_weights = MODEL_PATH+path
    tmp= tf.keras.models.model_from_json(loaded_model_json)
    tmp.load_weights(Model_weights)
    models.append(tmp)

In [ ]:
# reading images
import pandas as pd

In [6]:
fito=pd.read_excel('C:/Users/USUARIO/Desktop/Tesis/gitrepo/cocoa_dl_android/model_experimentation/Strings.xlsx',sheet_name="Fito",header=None,names=["strs"])

In [10]:
fito.columns

Index(['strs'], dtype='object')

In [11]:
fito.iloc[0]['strs']

'...'

In [9]:
fito.agg(lambda x: ', '.join(x.dropna())).to_frame().T

,strs
0,iVBORw0KGgoAAAANSUhEUgAAALsAAAC7CAIAAACy8ng9AA...


In [65]:
fito_string=r"""
...
"""

In [66]:
im_bytes = base64.b64decode(fito_string)   # im_bytes is a binary image
im_file = BytesIO(im_bytes)  # convert image to file-like object
img = Image.open(im_file)   # img is now PIL Image object
img=remove(img)
img = img.convert(mode='L')
img = img.convert(mode='RGB')
img = img.resize((300, 300))

In [14]:
img.save(r"C:\Users\USUARIO\Desktop\Tesis\gitrepo\cocoa_dl_android\model_experimentation\removed_img.png")

In [9]:
img.show()

In [61]:
img_path=r"D:\experimentos\MLFlowTesting\Imagenes para experimentos\Mazorca_SF\train\Mazorca_Negra\ima0.jpg"
#img_path=r"D:\experimentos\MLFlowTesting\Imagenes para experimentos\Mazorca_SF\train\Sanos\ima5.jpg"
img_removed= Image.open(img_path)
img_removed = img_removed.convert(mode='L')
img_removed = img_removed.convert(mode='RGB')
img_removed = img_removed.resize((300, 300))

In [67]:
x = tf.keras.utils.img_to_array(img)
x = np.true_divide(x, 255)
x = np.expand_dims(x, axis=0)
print ("preprocess compleated")

preprocess compleated


In [39]:
x.shape

(1, 300, 300, 3)

In [68]:
preds=[]
for model in models:
    individual_preds = model.predict(x)
    individual_preds=individual_preds.tolist()[0]
    preds.append(individual_preds[0]) 

1/1 [==============================] - 0s 16ms/step


In [69]:
preds

[0.5810385942459106,
 0.7545487284660339,
 0.06469528377056122,
 0.18128956854343414]

In [13]:
img.show()